## Example of the model application for the stock TROW

In [1]:
import pandas.util.testing as tm
from FunctionsINVCHA import *

ipykernel_launcher:1: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [2]:
#1) Create the dataframe
df = create_data("TROW")

#2) Tune hyperparameters for XGB
inputs = ['RSI', 'SR_signal', 'MACD', 'ATR', 'UD_SP', '10days_Vol']
output = "Signal"

tuned = tune_param_XGB([3,4,5,6,7,8,9,10], [1,2,3,4,5,6,7,8,9,10],                            data=df, inputs=inputs, output=output, pct_in=0.6)
tuned

[0.7546450690805145, 3, 10]

In [3]:
#3) Fit the models (in the real application the model is fitted with the entire dataset available at the time, using all_data=True in the models functions). 
# Only hyperparameters tuning (and the previous screening process) was performed after splitting in train and test data
mod_rf, accrf = random_forest_model(df, inputs=inputs, output=output, pct_in=0.70, n_trees=100)[0:2] 
mod_xgb, accxgb = xgboost_model(df, inputs=inputs, output=output,pct_in=0.70, max_depth=tuned[1], min_child_weight=tuned[2])[0:2] 
mod_lda, acclda = LDA_class(df, inputs=inputs, output=output, pct_in=0.70)[0:2] 
mod_qda, accqda = QDA_class(df, inputs=inputs, output=output, pct_in=0.70)[0:2] 

print("OOS Accuracy RF: ", accrf.round(3))
print("OOS Accuracy XGB: ", accxgb.round(3))
print("OOS Accuracy LDA: ", acclda.round(3))
print("OOS Accuracy QDA: ", accqda.round(3))

OOS Accuracy RF:  0.746
OOS Accuracy XGB:  0.758
OOS Accuracy LDA:  0.771
OOS Accuracy QDA:  0.748


In [6]:
# Get data for today and use to predict tomorrow direction
data_to_predict = compute_data_toadd("TROW")[1] 
X_to_predict = np.array(data_to_predict).reshape(1, -1)

In [8]:
final_output_rf = mod_rf.predict(X_to_predict)
final_output_xg = mod_xgb.predict(X_to_predict)
final_output_lda = mod_lda.predict(X_to_predict)
final_output_qda = mod_qda.predict(X_to_predict)

from datetime import date
today = date.today()
print("Today inputs: ", X_to_predict)
print(f"-------------- Predictions for tomorrow given today {today} data --------------")
print("Random Forest: ", "UP" if final_output_rf==1 else "DOWN")
print("XGBoost:       ", "UP" if final_output_xg==1 else "DOWN")
print("LDA:           ", "UP" if final_output_lda==1 else "DOWN")
print("QDA:           ", "UP" if final_output_qda==1 else "DOWN")
print("--------------------------------------------------------------------------------")

Today inputs:  [[2.93790346e+01 6.04442557e+01 2.32647113e+00 3.97710872e+00
  1.00000000e+00 5.28906516e-03]]
-------------- Predictions for tomorrow given today 2021-04-13 data --------------
Random Forest:  UP
XGBoost:        UP
LDA:            UP
QDA:            UP
--------------------------------------------------------------------------------


In [35]:
########## Example showing the process for each daily prediction ###########
def example_data_toadd(ticker, start_date, date_today):  #date
    data = web.DataReader(ticker, "yahoo", start=start_date, end=date_today)
    data = data.dropna()
    sp = web.DataReader("SPY", "yahoo", start=start_date, end=date_today)
    sp["Ret_SP"] = sp["Adj Close"].pct_change()
    sp["UD_SP"] = np.where((sp["Ret_SP"] > 0), 1, 0)
    sp["10days_Vol"] = sp["Ret_SP"].rolling(10).std()
    sp = sp.dropna()

    #RSI
    data["RSI"] = ta.momentum.rsi(close = data["Close"], n=2)
    #Stochastic Oscillator (I will use the signal as an input)
    data["SR_signal"] = ta.momentum.stoch_signal(close=data["Close"], 
                                                high=data["High"], low=data["Low"], n=5)
    #MACD (I will use the signal as an input, with default values for the two MA)
    data["MACD"] = ta.trend.macd_signal(close=data["Close"])
    #Average true range (ATR) for 5-days period
    data["ATR"] = ta.volatility.average_true_range(high=data["High"], 
                                                low=data["Low"], close=data["Close"], n=5)
        
    data = data.merge(sp[["UD_SP", "10days_Vol"]], on="Date")

    return data, data[["RSI", "SR_signal", "MACD", "ATR", "UD_SP", "10days_Vol"]].loc[date_today]

In [36]:
tick = "TROW"
mod_forest = pickle.load(open(f"modelsML/modelRF_{tick}.pkl", "rb"))
mod_xgb = pickle.load(open(f"modelsML/modelXGB_{tick}.pkl", "rb"))
mod_lda = pickle.load(open(f"modelsML/modelLDA_{tick}.pkl", "rb"))
mod_qda = pickle.load(open(f"modelsML/modelQDA_{tick}.pkl", "rb"))

In [38]:
dates_list = pd.date_range(start="2021-03-15", end="2021-04-13")

for date in dates_list:
    try:
        data_to_predict = example_data_toadd("TROW", start_date="2020-11-01", date_today=date)[1] 
        X_to_predict = np.array(data_to_predict).reshape(1, -1)
        final_output_rf = mod_rf.predict(X_to_predict)
        final_output_xg = mod_xgb.predict(X_to_predict)
        final_output_lda = mod_lda.predict(X_to_predict)
        final_output_qda = mod_qda.predict(X_to_predict)
        print(f"-------------- Predictions with {date} data --------------")
        # it means that we are using data for this date to predict for tomorrow
        print("INPUTS: ", X_to_predict)
        print("Random Forest: ", "UP" if final_output_rf==1 else "DOWN")
        print("XGBoost:       ", "UP" if final_output_xg==1 else "DOWN")
        print("LDA:           ", "UP" if final_output_lda==1 else "DOWN")
        print("QDA:           ", "UP" if final_output_qda==1 else "DOWN")
        print("---------------------------------------------------------------------")
    except KeyError:
        print("no date")

-------------- Predictions with 2021-03-15 00:00:00 data --------------
INPUTS:  [[2.70319871e+01 4.44170868e+01 3.51224994e+00 4.90828593e+00
  1.00000000e+00 1.10718316e-02]]
Random Forest:  UP
XGBoost:        UP
LDA:            UP
QDA:            UP
---------------------------------------------------------------------
-------------- Predictions with 2021-03-16 00:00:00 data --------------
INPUTS:  [[8.98463167e+00 3.09372303e+01 3.55571955e+00 4.54662691e+00
  0.00000000e+00 1.06260182e-02]]
Random Forest:  DOWN
XGBoost:        DOWN
LDA:            DOWN
QDA:            DOWN
---------------------------------------------------------------------
-------------- Predictions with 2021-03-17 00:00:00 data --------------
INPUTS:  [[7.46447526e+01 4.12490647e+01 3.58832750e+00 4.39129933e+00
  1.00000000e+00 9.08646897e-03]]
Random Forest:  DOWN
XGBoost:        UP
LDA:            UP
QDA:            UP
---------------------------------------------------------------------
-------------- Predic